In [1]:
import lsdb
import numpyro
import numpyro.distributions as dist
import numpy as np
import jax.numpy as jnp
import jax
import nested_pandas as npd

In [2]:
df = npd.read_parquet("data/lightcurve_thindisk_fixed_nested.parquet")
cols = [c for c in df.columns if c not in ["transfer_function"]]
df = df[cols]
df

ID  cos_inc  log_mbh  log_mdot  redshift  \
0    0    0.866      8.0       0.0       0.1   
1    1    0.866      8.0       0.0       0.1   
..  ..      ...      ...       ...       ...   
98  98    0.866      8.0       0.0       0.1   
99  99    0.866      8.0       0.0       0.1   

                                                  lag  \
0   [2.18424467 2.97357418 4.20083785 5.36442491 6...   
1   [2.18424467 2.97357418 4.20083785 5.36442491 6...   
..                                                ...   
98  [2.18424467 2.97357418 4.20083785 5.36442491 6...   
99  [2.18424467 2.97357418 4.20083785 5.36442491 6...   

                                          light_curve  
0   [{time: 0.0, band: 0, mag: 18.601379}; …] (120...  
1   [{time: 0.0, band: 0, mag: 19.047684}; …] (120...  
..                                                ...  
98  [{time: 0.0, band: 0, mag: 19.9523}; …] (1200 ...  
99  [{time: 0.0, band: 0, mag: 19.707044}; …] (120...  

[100 rows x 7 columns]

In [3]:
test_cat = lsdb.generate_catalog(len(df), 0)
test_cat

,ra,dec,id,a,b,nested
npartitions=12,,,,,,
"Order: 0, Pixel: 0",double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],"nested<t: [double], flux: [double], band: [nul..."
"Order: 0, Pixel: 1",...,...,...,...,...,...
...,...,...,...,...,...,...
"Order: 0, Pixel: 10",...,...,...,...,...,...
"Order: 0, Pixel: 11",...,...,...,...,...,...


In [4]:
test_cat_comp = test_cat.compute()

In [5]:
df["ra"] = test_cat_comp["ra"].to_numpy()
df["dec"] = test_cat_comp["dec"].to_numpy()
df

ID  cos_inc  log_mbh  log_mdot  redshift  \
0    0    0.866      8.0       0.0       0.1   
1    1    0.866      8.0       0.0       0.1   
..  ..      ...      ...       ...       ...   
98  98    0.866      8.0       0.0       0.1   
99  99    0.866      8.0       0.0       0.1   

                                                  lag  \
0   [2.18424467 2.97357418 4.20083785 5.36442491 6...   
1   [2.18424467 2.97357418 4.20083785 5.36442491 6...   
..                                                ...   
98  [2.18424467 2.97357418 4.20083785 5.36442491 6...   
99  [2.18424467 2.97357418 4.20083785 5.36442491 6...   

                                          light_curve          ra        dec  
0   [{time: 0.0, band: 0, mag: 18.601379}; …] (120...   49.129985  18.191942  
1   [{time: 0.0, band: 0, mag: 19.047684}; …] (120...   11.058632  82.110192  
..                                                ...         ...        ...  
98  [{time: 0.0, band: 0, mag: 19.9523}; …] (1200 ...  285.675099 -57.662798  
99  [{time: 0.0, band: 0, mag: 19.707044}; …] (120...  346.488226 -54.303029  

[100 rows x 9 columns]

In [6]:
cat = lsdb.from_dataframe(df)
cat

,ID,cos_inc,log_mbh,log_mdot,redshift,lag,light_curve,ra,dec
npartitions=12,,,,,,,,,
"Order: 0, Pixel: 0",int64[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],list<element: double>[pyarrow],"nested<time: [double], band: [int64], mag: [do...",double[pyarrow],double[pyarrow]
"Order: 0, Pixel: 1",...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
"Order: 0, Pixel: 10",...,...,...,...,...,...,...,...,...
"Order: 0, Pixel: 11",...,...,...,...,...,...,...,...,...


In [7]:
from eztaox.kernels.quasisep import Exp
from eztaox.fitter import random_search
from eztaox.models import MultiVarModel

SNR = 1000.0


def run_eztaox(time, band, mag):
    times, mags, noisy_mags, mag_errs = {}, {}, {}, {}
    bands = list(np.unique(band)[[0, -1]])
    for b in bands:
        mask = band == b
        times[b] = time[mask]
        mags[b] = mag[mask]
        noisy_mags[b] = (
            mags[b] + np.random.normal(0, 1, size=mags[b].shape) * mags[b] / SNR
        )
        mag_errs[b] = mags[b] / SNR
    inds = jnp.argsort(jnp.concatenate([times[b] for b in bands]))
    X = (
        jnp.concatenate([times[b] for b in bands])[inds],
        jnp.concatenate(
            [i * jnp.ones_like(times[b], dtype=int) for i, b in enumerate(bands)]
        )[inds],
    )
    for b in bands:
        noisy_mags[b] = jnp.array(noisy_mags[b])
        noisy_mags[b] -= jnp.median(noisy_mags[b])

    y = jnp.concatenate([noisy_mags[b] for b in bands])[inds]
    yerr = jnp.concatenate([mag_errs[b] for b in bands])[inds]

    has_lag = True  # if fit interband lags
    zero_mean = True  # if fit a mean function
    nBand = len(bands)

    # initialize a GP kernel, note the initial parameters are not used in the fitting
    k = Exp(scale=100.0, sigma=1.0)
    m = MultiVarModel(X, y, yerr, k, nBand, has_lag=has_lag, zero_mean=zero_mean)

    def initSampler():
        # GP kernel param
        log_drw_scale = numpyro.sample(
            "drw_scale", dist.Uniform(jnp.log(0.01), jnp.log(1000))
        )
        log_drw_sigma = numpyro.sample(
            "drw_sigma", dist.Uniform(jnp.log(0.01), jnp.log(10))
        )
        log_kernel_param = jnp.stack([log_drw_scale, log_drw_sigma])
        numpyro.deterministic("log_kernel_param", log_kernel_param)

        # parameters to relate the amplitudes in each band
        log_amp_scale = numpyro.sample("log_amp_scale", dist.Uniform(-2, 2))

        mean = numpyro.sample(
            "mean",
            dist.Uniform(low=jnp.asarray([-0.1, -0.1]), high=jnp.asarray([0.1, 0.1])),
        )

        # interband lags
        lag = numpyro.sample("lag", dist.Uniform(-10, 10))

        sample_params = {
            "log_kernel_param": log_kernel_param,
            "log_amp_scale": log_amp_scale,
            "mean": mean,
            "lag": lag,
        }

        return sample_params

    model = m
    fit_key = jax.random.PRNGKey(1)
    nSample = 1_000
    nBest = 5  # it seems like this number needs to be high

    bestP, ll = random_search(model, initSampler, fit_key, nSample, nBest)

    return {"best_params": bestP, "log_likelihood": ll}

In [8]:
res = df.iloc[:10].map_rows(
    run_eztaox,
    columns=["light_curve.time", "light_curve.band", "light_curve.mag"],
    row_container="args",
)
res

,best_params,log_likelihood
0,"{'lag': 4.4427317241994855, 'log_amp_scale': -0.016276631787436466, 'log_kernel_param': [6.279124152510852, -0.31287537159028994], 'mean': [-0.08006180995604205, 0.05387841598852514]}",644.3022278270316
1,"{'lag': 2.5220236236155498, 'log_amp_scale': -0.04781707883644922, 'log_kernel_param': [2.624644753701827, -1.1282042179526635], 'mean': [-0.0013556186603485899, 0.004095973980314672]}",387.6282536975185
2,"{'lag': 2.546739111595922, 'log_amp_scale': -0.011434638750367833, 'log_kernel_param': [3.878538791187697, -0.6168841602141807], 'mean': [-0.0013556186603485899, 0.004095973980314672]}",412.1457257356112
3,"{'lag': 3.4939982361526862, 'log_amp_scale': -0.09616413077378716, 'log_kernel_param': [2.717056769296256, -1.4989488871008196], 'mean': [0.0020413791618782984, -0.03800461300923348]}",517.5763808383691
4,"{'lag': 2.5391989044006427, 'log_amp_scale': -0.04892972846764508, 'log_kernel_param': [3.845304600500822, -1.0319373310809847], 'mean': [-0.03511726390175549, 0.03982690277189787]}",534.6033382838543
5,"{'lag': 3.5059476508113376, 'log_amp_scale': -0.1082631551911636, 'log_kernel_param': [3.208688616157533, -1.3874193183288317], 'mean': [0.0020413791618782984, -0.03800461300923348]}",558.2529262298317
6,"{'lag': 3.534200049318459, 'log_amp_scale': -0.1721156483530428, 'log_kernel_param': [3.816869169711987, -1.6930847766490025], 'mean': [0.0020413791618782984, -0.03800461300923348]}",711.5022424647647
7,"{'lag': 4.4554262659110435, 'log_amp_scale': -0.06471678668693007, 'log_kernel_param': [4.190615610199421, -0.9163469388711006], 'mean': [0.08278969359306863, -0.05068042678406135]}",558.9005014873131
8,"{'lag': 2.4914984708851367, 'log_amp_scale': -0.020576717040206, 'log_kernel_param': [4.267119122763998, -0.4858243765930996], 'mean': [-0.0013556186603485899, 0.004095973980314672]}",433.82622059924756
9,"{'lag': 4.452736288654676, 'log_amp_scale': -0.05250372984764836, 'log_kernel_param': [3.951057462292397, -1.2294084398955423], 'mean': [0.08278969359306863, -0.05068042678406135]}",606.5755321589978


In [9]:
from distributed import Client

try:
    client.close()
    client.shutdown()
except:
    pass

client = Client(n_workers=4)

/Users/smcmu/.pyenv/versions/eztaox/lib/python3.12/site-packages/distributed/node.py:188: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55283 instead
  warnings.warn(


In [10]:
res = cat.map_rows(
    run_eztaox,
    columns=["light_curve.time", "light_curve.band", "light_curve.mag"],
    row_container="args",
    meta={"best_params": "object", "log_likelihood": "float64"},
).compute()
res

2026-02-17 18:32:09,722 - distributed.worker - ERROR - Compute Failed
Key:       ('reduce_part-665a9fb98e36826a954060559f2649fc', 7)
State:     executing
Task:  <Task ('reduce_part-665a9fb98e36826a954060559f2649fc', 7) apply_and_enforce(..., ...)>
Exception: "Exception('Long error message', <class 'jax.errors.UnexpectedTracerError'>, 'Encountered an unexpected tracer. A function transformed by JAX had a side effect, allowing for a reference to an intermediate value with type float64[] wrapped in a BatchTracer to escape the scope of the transformation.\\nJAX transformations require that functions explicitly return their outputs, and disallow saving intermediate values to global state.\\nTo catch the leak earlier, try setting the environment variable JAX_CHECK_TRACER_LEAKS or using the `jax.checking_leaks` context manager.Detail: Different traces at same level: Traced<ShapedArray(float64[])>with<BatchTrace(level=1/0)> with\\n  val = Array([ 5.97406971e-01, -9.87096458e+00,  5.85414131e+0

Exception: ('Long error message', <class 'jax.errors.UnexpectedTracerError'>, 'Encountered an unexpected tracer. A function transformed by JAX had a side effect, allowing for a reference to an intermediate value with type float64[] wrapped in a BatchTracer to escape the scope of the transformation.\nJAX transformations require that functions explicitly return their outputs, and disallow saving intermediate values to global state.\nTo catch the leak earlier, try setting the environment variable JAX_CHECK_TRACER_LEAKS or using the `jax.checking_leaks` context manager.Detail: Different traces at same level: Traced<ShapedArray(float64[])>with<BatchTrace(level=1/0)> with\n  val = Array([ 5.97406971e-01, -9.87096458e+00,  5.85414131e+00, -6.52926438e+00,\n       -4.93847830e+00, -3.02433360e+00, -5.61592785e+00,  3.08040436e+00,\n       -7.46335223e+00, -1.78246390e+00,  9.26314454e+00,  9.59303266e+00,\n        3.20983332e+00, -1.15491713e+00,  6.30826897e+00,  6.00515274e+00,\n        1.78

2026-02-17 18:32:11,101 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('reduce_part-665a9fb98e36826a954060559f2649fc', 2))" coro=<Worker.execute() done, defined at /Users/smcmu/.pyenv/versions/eztaox/lib/python3.12/site-packages/distributed/worker_state_machine.py:3608>> ended with CancelledError
2026-02-17 18:32:11,102 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('reduce_part-665a9fb98e36826a954060559f2649fc', 3))" coro=<Worker.execute() done, defined at /Users/smcmu/.pyenv/versions/eztaox/lib/python3.12/site-packages/distributed/worker_state_machine.py:3608>> ended with CancelledError
2026-02-17 18:32:11,104 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('reduce_part-665a9fb98e36826a954060559f2649fc', 4))" coro=<Worker.execute() done, defined at /Users/smcmu/.pyenv/versions/eztaox/lib/python3.12/site-packages/distributed/work

In [14]:
df["lag"].iloc[:10]

0    [2.18424467 2.97357418 4.20083785 5.36442491 6...
1    [2.18424467 2.97357418 4.20083785 5.36442491 6...
2    [2.18424467 2.97357418 4.20083785 5.36442491 6...
3    [2.18424467 2.97357418 4.20083785 5.36442491 6...
4    [2.18424467 2.97357418 4.20083785 5.36442491 6...
5    [2.18424467 2.97357418 4.20083785 5.36442491 6...
6    [2.18424467 2.97357418 4.20083785 5.36442491 6...
7    [2.18424467 2.97357418 4.20083785 5.36442491 6...
8    [2.18424467 2.97357418 4.20083785 5.36442491 6...
9    [2.18424467 2.97357418 4.20083785 5.36442491 6...
Name: lag, dtype: list<element: double>[pyarrow]

In [15]:
res.map_rows(
    lambda x: {
        "lag": x["best_params"]["lag"],
        "log_kernel_param": x["best_params"]["log_kernel_param"],
    }
)

,lag,log_kernel_param
0,4.485634800377804,"[6.280977131400453, -0.3222664536000996]"
1,2.52834132302178,"[3.0195855958455597, -1.1381191480470858]"
2,2.5234785598652016,"[3.4904263502785318, -0.6709334454384429]"
3,3.48099408975768,"[3.434506166367917, -1.499840031767058]"
4,2.542600098457406,"[3.8905857997133064, -1.028932612590522]"
5,3.4997980549832777,"[3.1631299820683223, -1.3626185410366085]"
6,3.4939133619599896,"[3.9096803848202293, -1.7146598255317873]"
7,3.53368758467303,"[3.8726327022708205, -0.9915142049711875]"
8,2.4899426259643582,"[4.325341505010771, -0.48920224520760264]"
9,4.465401841312857,"[3.9615853978492, -1.2280408477747586]"
